In [ ]:
json_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'

In [1]:
import requests
from requests import Response
def get_response() -> Response | None:
    json_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'
    try:
        response:Response =requests.get(json_url)
        response.raise_for_status()
        if response.status_code == 200:
            print("下載成功")
            return response
        else:
            print("下載失敗")
            return None
    except Exception as e:
        print(e)
        print("連線失敗")
        return None

In [ ]:
response:Response | None = get_response()
if response:
    rawString:str = response.text
    print(rawString)

In [3]:
from pydantic import BaseModel,Field,field_validator

class Site(BaseModel):
    站點名稱:str = Field(alias='sitename')
    縣市:str = Field(alias='county')
    aqi:float
    時間:str = Field(alias='datacreationdate')
    pm25:float = Field(alias='pm2.5')

    @field_validator("pm25",mode='before')
    @classmethod
    def empty_to_zero(cls,value):
        if value=='':
            return '0.0'
        else:
            return value
        
class Sites(BaseModel):
    records:list[Site]

In [ ]:
data:Sites = Sites.model_validate_json(rawString)

for item in data.records:
    print(item)

In [ ]:
download_data:list[dict] = data.model_dump()['records']
download_data

In [9]:
import pandas as pd
pd.DataFrame(download_data)

,站點名稱,縣市,aqi,時間,pm25
0,屏東(枋山),屏東縣,34.0,2024-03-28 20:00,7.0
1,新北(樹林),新北市,67.0,2024-03-28 20:00,18.0
2,屏東(琉球),屏東縣,40.0,2024-03-28 20:00,10.0
3,臺南(麻豆),臺南市,59.0,2024-03-28 20:00,17.0
4,高雄(湖內),高雄市,44.0,2024-03-28 20:00,7.0
...,...,...,...,...,...
995,萬華,臺北市,48.0,2024-03-28 09:00,22.0
996,中山,臺北市,43.0,2024-03-28 09:00,16.0
997,士林,臺北市,40.0,2024-03-28 09:00,20.0
998,淡水,新北市,46.0,2024-03-28 09:00,21.0
